# Setup

In [99]:
# Imports
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, TweetTokenizer, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import string
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
!pip install unidecode
from unidecode import unidecode
# pip install Counter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.cm as cm
from sklearn.decomposition import TruncatedSVD
import csv

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\evano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\evano\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\evano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [113]:
root = 'Data/'

In [101]:
# Decompress files and concatenate to datframe
filenames = ['UkraineCombinedTweetsDeduped_FEB27.csv.gzip',
             'UkraineCombinedTweetsDeduped_FEB28_part1.csv.gzip',
             'UkraineCombinedTweetsDeduped_FEB28_part2.csv.gzip',
             'UkraineCombinedTweetsDeduped_MAR01.csv.gzip',
             'UkraineCombinedTweetsDeduped_MAR02.csv.gzip',
             'UkraineCombinedTweetsDeduped_MAR03.csv.gzip',
             'UkraineCombinedTweetsDeduped_MAR04.csv.gzip',
             'UkraineCombinedTweetsDeduped_MAR05.csv.gzip']
  
li = []

for filename in filenames:
    df = pd.read_csv(root + filename, 
                     index_col=None, 
                     header=0,
                     compression='gzip')
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)

C:\Users\evano\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Cleaning & Tokenizing

In [102]:
n = 10000

In [103]:
data.shape

(2984341, 18)

In [104]:
# Define function to clean text (to lowercase, remove punctuation, extra whitespace)
hashtags = ['#slavaukraini','#russia','#russiaukraineWar','#putin','#russiaukraine', '#russianwar', '#ww3', '#moscow', 
            '#russianconflict', '#ukraineunderattack', '#ukriane', '#ukraine', '#russianukrainianWar', '#ukrainerussia', 
            '#ukraineconflict', '#ukrainewar', '#kharkiv', '#stopputinNow']

def clean_text(text):
    text_nonum = re.sub(r'\d+', '', text)
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in '!"$%&\'()*+,-./:;<=>?[\]^_`{|}~']) 
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    
    return text_no_doublespace

# Define function to determine if a token is junk (@'s, URLs, unwanted hashtags)
def is_junk(token):
  if ('https' in token) or ('@' in token) or (len(token) <= 2) or (token in hashtags):
    return False
  else:
    return True

# Define function to drop unwanted tokens
def drop_tokens(tokens):
  return list(filter(lambda token: is_junk(token), tokens))

# Lemmatize words to get each word to its root
def lemmatize_words(words):
  lemmatizer = WordNetLemmatizer()
  lem_words = [lemmatizer.lemmatize(w) for w in words]
  return lem_words

In [105]:
# Remove any tweets not in English
data = data.loc[data['language'] == 'en']
data.shape

(1961759, 18)

In [106]:
# Clean text
data.text = data.text.apply(clean_text)

# Tokenize text
tt = TweetTokenizer()
tokenized_text = data.text.apply(tt.tokenize)

In [107]:
# Drop bad tokens
tokenized_text = tokenized_text.apply(drop_tokens)

In [108]:
# Apply lemmetization
tokenized_text = tokenized_text.apply(lemmatize_words)

In [109]:
tokenized_text.head()

0    [#ukrainerussiawar, captured, russian, soldier...
1    [like, everybody, else, rooting, for, ukraine,...
3    [elected, game, show, host, and, got, clown, e...
4    [ukrainian, soldier, wearing, sunflower, their...
5    [russian, troop, destroyed, commercial, oil, b...
Name: text, dtype: object

In [114]:
# Write output file
with open(root + 'cleaned_tokenized_data.csv', mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    for row in tokenized_text:
        writer.writerow(row)